In [8]:
import pandas as pd
import warnings

from zmq.utils.strtypes import unicode

data = pd.read_csv("emails.csv")

# print(data.columns)
# print(data.head(2))

In [9]:
def extract_body(email_text):
    if isinstance(email_text, str):
        parts = email_text.split("\n\n", 1)
        if len(parts) > 1:
            return parts[1]
    return ""

In [11]:
data["clean_text"] = data["message"].apply(extract_body)
data.info

<bound method DataFrame.info of                              file  \
0           allen-p/_sent_mail/1.   
1          allen-p/_sent_mail/10.   
2         allen-p/_sent_mail/100.   
3        allen-p/_sent_mail/1000.   
4        allen-p/_sent_mail/1001.   
...                           ...   
517396  zufferli-j/sent_items/95.   
517397  zufferli-j/sent_items/96.   
517398  zufferli-j/sent_items/97.   
517399  zufferli-j/sent_items/98.   
517400  zufferli-j/sent_items/99.   

                                                  message  \
0       Message-ID: <18782981.1075855378110.JavaMail.e...   
1       Message-ID: <15464986.1075855378456.JavaMail.e...   
2       Message-ID: <24216240.1075855687451.JavaMail.e...   
3       Message-ID: <13505866.1075863688222.JavaMail.e...   
4       Message-ID: <30922949.1075863688243.JavaMail.e...   
...                                                   ...   
517396  Message-ID: <26807948.1075842029936.JavaMail.e...   
517397  Message-ID: <25835861.10758

In [25]:
def extract_body(email_text):
    if isinstance(email_text, str):
        parts = email_text.split("\n\n", 1)
        if len(parts) > 1:
            return parts[1]
    return ""

data["clean_text"] = data["message"].apply(extract_body)

data["label"] = 0
data_cleaned = data[["clean_text", "label"]]
data_cleaned.columns = ["text", "label"]
data_cleaned = data_cleaned[data_cleaned["text"].str.strip() != ""]
data_cleaned.head(2)
data_cleaned = data_cleaned.sample(n=5500, random_state=42)
data_cleaned.to_csv("emails_cleaned_ham.csv", index=False)
data_cleaned.info()

<class 'pandas.DataFrame'>
Index: 5500 entries, 427616 to 361829
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   text    5500 non-null   str  
 1   label   5500 non-null   int64
dtypes: int64(1), str(1)
memory usage: 128.9 KB


In [26]:
import glob
import pandas as pd

spam_texts = []

for file in glob.glob("spam_2/spam_2/*"):
    with open(file, encoding="latin-1") as f:
        spam_texts.append(f.read())

data_spam = pd.DataFrame({
    "text": spam_texts,
    "label": 1
})

print("Spam emails:", data_spam.shape)
print(data_spam.head(2))

def extract_body(email_text):
    if isinstance(email_text, str):
        parts = email_text.split("\n\n", 1)
        if len(parts) > 1:
            return parts[1]
    return ""

data_spam["text"] = data_spam["text"].apply(extract_body)
data_spam = data_spam[data_spam["text"].str.strip() != ""]
data_spam.to_csv("spam_cleaned_spam.csv", index=False)
data_spam.info()
data_spam.head(2)
data_spam.info()

Spam emails: (1397, 2)
                                                text  label
0  From ilug-admin@linux.ie  Tue Aug  6 11:51:02 ...      1
1  From lmrn@mailexcite.com  Mon Jun 24 17:03:24 ...      1
<class 'pandas.DataFrame'>
RangeIndex: 1396 entries, 0 to 1395
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   text    1396 non-null   str  
 1   label   1396 non-null   int64
dtypes: int64(1), str(1)
memory usage: 21.9 KB
<class 'pandas.DataFrame'>
RangeIndex: 1396 entries, 0 to 1395
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   text    1396 non-null   str  
 1   label   1396 non-null   int64
dtypes: int64(1), str(1)
memory usage: 21.9 KB


In [34]:
import pandas as pd

ham_data = pd.read_csv("emails_cleaned_ham.csv")
spam_data = pd.read_csv("spam_cleaned_spam.csv")

print(ham_data.shape)
print(spam_data.shape)

print(ham_data.columns)
print(spam_data.columns)

final_data = pd.concat([ham_data, spam_data], ignore_index=True)

final_data = final_data.sample(frac=1, random_state=42).reset_index(drop=True)

final_data.info()
final_data["label"].value_counts()

final_data.to_csv("final_dataset.csv", index=False)

(5500, 2)
(1396, 2)
Index(['text', 'label'], dtype='str')
Index(['text', 'label'], dtype='str')
<class 'pandas.DataFrame'>
RangeIndex: 6896 entries, 0 to 6895
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   text    6896 non-null   str  
 1   label   6896 non-null   int64
dtypes: int64(1), str(1)
memory usage: 107.9 KB


In [39]:
import re
from bs4 import BeautifulSoup, MarkupResemblesLocatorWarning
import warnings

warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)

def clean_email(text):
    if not isinstance(text, str):
        return ""

    # Remove URLs FIRST
    text = re.sub(r"http\S+|www\S+", " ", text)

    # Remove email addresses
    text = re.sub(r"\S+@\S+", " ", text)

    # Remove MIME headers
    text = re.sub(r"Content-Type:.*", " ", text)
    text = re.sub(r"Content-Transfer-Encoding:.*", " ", text)
    text = re.sub(r"charset=.*", " ", text)

    # Remove MIME boundaries
    text = re.sub(r"-{2,}.*", " ", text)

    # Remove HTML tags safely
    if "<" in text and ">" in text:
        text = BeautifulSoup(text, "html.parser").get_text()

    # Normalize whitespace
    text = re.sub(r"\s+", " ", text).strip()

    return text

data = pd.read_csv("final_dataset.csv")
data["text"] = data["text"].apply(clean_email)
data = data[data["text"].str.strip() != ""]

data.to_csv("final_dataset_new.csv", index=False)


In [7]:
import pandas as pd

data = pd.read_csv("spam_email_dataset.csv", encoding="latin-1")
# data.head(10)

# remove the literal word 'SPA'
data["text"] = data["text"].str.replace("SPA", "", regex=False)

# remove extra spaces created after removal
data["text"] = data["text"].str.replace(r"\s+", " ", regex=True).str.strip()

data.head(10)




,text,label
0,Exclusive Loan Offer  Get Instant Approval,1
1,Exclusive Loan Offer  Get Instant Approval,1
2,"Win $1,000,000 Now  Limited Time Offer!",1
3,Act Fast! Your Netflix Subscription Is Expiring,1
4,Hot Investment Alert  10x Your Money!,1
5,Warning! Your Computer Is Infected  Fix Now!,1
6,Congratulations! You Have Won a Free iPhone!,1
7,Exclusive Loan Offer  Get Instant Approval,1
8,"Win $1,000,000 Now  Limited Time Offer!",1
9,Security Alert! Your Amazon Account Is Locked,1


In [10]:
import pandas as pd

spam1_data = pd.read_csv("spam_email_dataset.csv", encoding="latin-1")
spam_data = pd.read_csv("final_dataset_new.csv")

print(spam1_data.shape)
print(spam_data.shape)

print(spam1_data.columns)
print(spam_data.columns)

final_data = pd.concat([spam1_data, spam_data], ignore_index=True)

final_data = final_data.sample(frac=1, random_state=42).reset_index(drop=True)

final_data.info()
final_data["label"].value_counts()

final_data.to_csv("final_final_dataset.csv", index=False)

(1000, 2)
(6876, 2)
Index(['text', 'label'], dtype='str')
Index(['text', 'label'], dtype='str')
<class 'pandas.DataFrame'>
RangeIndex: 7876 entries, 0 to 7875
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   text    7876 non-null   str  
 1   label   7876 non-null   int64
dtypes: int64(1), str(1)
memory usage: 123.2 KB
